In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import copy
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
!wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/test.csv
!wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/train.csv

--2023-07-23 06:57:40--  https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89724 (88K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  87.62K  --.-KB/s    in 0.02s   

2023-07-23 06:57:40 (5.38 MB/s) - ‘test.csv’ saved [89724/89724]

--2023-07-23 06:57:40--  https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791227 (773K) [text/

In [3]:
train_set = pd.read_csv("/content/train.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)

test_set = pd.read_csv("/content/test.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)


In [4]:
train_set.head()

,text,category
0,Trailer late ah parthavanga like podunga,Positive
1,Move pathutu vanthu trailer pakurvnga yaru,Positive
2,Puthupetai dhanush ah yarellam pathinga,Positive
3,"Dhanush oda character ,puthu sa erukay , mass ta",Positive
4,vera level ippa pesungada mokka nu thalaivaaaaaa,Positive


In [5]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1260 non-null   object
 1   category  1260 non-null   object
dtypes: object(2)
memory usage: 19.8+ KB


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [7]:
def take_data_to_shower(tweet):
    tweet=str(tweet)

    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)

In [8]:
def remove_emoticon(text):
        text=str(text)

        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

In [9]:
def remove_punc(text):
    text=str(text)
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [10]:
def tokenize(tweet):
   tweet=str(tweet)

   lower_tweet = tweet.lower()
   return word_tokenize(lower_tweet)

In [11]:
def remove_stop_words(tokens):
   clean_tokens = []
   newStopWords = ['rt', 'mkr', 'didn', 'bc', 'n', 'm', 'im', 'll', 'y', 've', 'u', 'ur', 'don', 't', 's', 'co','oru', 'enru', 'matrum', 'indha', 'idhu', 'enru', 'kondu', 'enpathu', 'pala', 'aagum', 'allathu', 'avar', 'naan', 'ullu', 'andha', 'ivar', 'ena', 'muthal', 'enna', 'irundhu', 'sila', 'en', 'pontu', 'venṭum', 'vandhu', 'ithan', 'athu', 'avan', 'thaan', 'palarum', 'ennum', 'melum', 'pinar', 'kondu', 'irukkum', 'thanathu', 'ullathu', 'pothu', 'enrum', 'athan', 'than', 'piragu', 'avargal', 'varai', 'aval', 'nee', 'aagiyathu', 'ullana', 'vandha', 'irundh', 'migavum', 'inggu', 'meethu', 'or', 'ivai', 'indha', 'ithaṉaḷ', 'patri', 'varum', 'veṟu', 'iru', 'ithil', 'pol', 'ippothu', 'avarathu', 'maṭṭum', 'indha', 'enuṁ', 'mel', 'pin', 'cernta', 'ākiyōr', 'enakku', 'innum', 'andha', 'anṟu', 'oṟe', 'miga', 'angu', 'palvēru', 'viṭṭu', 'perum', 'athai', 'paṟṟiya', 'uṉ', 'athik', 'andhak', 'pēr', 'indhaṉāḷ', 'avai', 'athe', 'ēn', 'muṟai', 'yār', 'eṉappaṭum', 'ellām', 'maṭṭumē', 'ingkē', 'angkē', 'iṭam', 'iṭattil', 'athil', 'nām', 'atharku', 'eṉavē', 'piṟa', 'siṟu', 'maṟṟa', 'viṭa', 'endha', 'eṉavum', 'eṉappuṭum', 'eṉiṉum', 'aṭutta', 'ithanai', 'ithai', 'koḷḷa', 'indhath', 'ithaṉāl', 'athāl', 'thavira', 'pol', 'varail', 'caṟṟu', 'eṉak']
   stopWords = stopwords.words('english')
   stopWords.append(newStopWords)
  #  print(stopWords)

   for token in tokens:
       if token not in stopWords:
           if token.replace(' ', '') != '':
               if len(token) > 1:
                   clean_tokens.append(token)
   return clean_tokens

In [12]:
def stem_and_lem(tokens):
   clean_tokens = []
   for token in tokens:
       token = wordnet_lemmatizer.lemmatize(token)
       token = lancaster_stemmer.stem(token)
       if len(token) > 1:
           clean_tokens.append(token)
   return ' '.join(clean_tokens)


In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [14]:
##EMBEDDING##
def datacleen(data):
  tqdm.pandas(desc="Remove duplicates ")
  data.drop_duplicates(subset=['text'], inplace=True)
  tqdm.pandas(desc="To lower letter")
  data = data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

  tqdm.pandas(desc="Cleaning Data Phase I...")
  data['take_data_to_shower'] = data['text'].progress_apply(take_data_to_shower)

  tqdm.pandas(desc="removing emoji...")
  data['remove_emoticon'] = data['take_data_to_shower'].progress_apply(remove_emoticon)


  tqdm.pandas(desc="removing punctuation...")
  data['remove_punc'] = data['remove_emoticon'].progress_apply(remove_punc)



  tqdm.pandas(desc="Tokenizing Data...")
  data['tokenize'] = data['remove_punc'].progress_apply(tokenize)

  tqdm.pandas(desc="Cleaning Data Phase II...")
  data['remove_stop_words'] = data['tokenize'].progress_apply(remove_stop_words)

  tqdm.pandas(desc="Stemming And Lemmatizing")
  data['stem_and_lem'] = data['remove_stop_words'].progress_apply(stem_and_lem)
  tqdm.pandas(desc="Cleaning Data Phase III...")

  # data.drop(data[(data['category'] == 'unknown_state') | (data['category'] == 'not-tamil') |(data['category'] == 'non-tamil') |  (data['category'] == 'mixed_feelings')].index, inplace=True)
  data.drop(data[(data['category'] == 'mixed_feelings')].index, inplace=True)
  # replacement_dict = {
  #     'positive': 0,
  #     'negative': 1,
  #     'mixed_feelings':-1,
  # }
  # # Replace values in the DataFrame
  # data['category'].replace(replacement_dict, inplace=True)


  # stem_and_lem ---> outlet
  return data
  # text_vector = clean_texts['tokens'].tolist()


In [15]:
train_set = datacleen(train_set)
train_set

Stemming And Lemmatizing: 100%|██████████| 11237/11237 [00:03<00:00, 3412.27it/s]


,text,category,take_data_to_shower,remove_emoticon,remove_punc,tokenize,remove_stop_words,stem_and_lem
0,trailer late ah parthavanga like podunga,positive,trailer late ah parthavanga like podunga,trailer late ah parthavanga like podunga,trailer late ah parthavanga like podunga,"[trailer, late, ah, parthavanga, like, podunga]","[trailer, late, ah, parthavanga, like, podunga]",trail lat ah parthavang lik podung
1,move pathutu vanthu trailer pakurvnga yaru,positive,move pathutu vanthu trailer pakurvnga yaru,move pathutu vanthu trailer pakurvnga yaru,move pathutu vanthu trailer pakurvnga yaru,"[move, pathutu, vanthu, trailer, pakurvnga, yaru]","[move, pathutu, vanthu, trailer, pakurvnga, yaru]",mov pathutu vanthu trail pakurvng yaru
2,puthupetai dhanush ah yarellam pathinga,positive,puthupetai dhanush ah yarellam pathinga,puthupetai dhanush ah yarellam pathinga,puthupetai dhanush ah yarellam pathinga,"[puthupetai, dhanush, ah, yarellam, pathinga]","[puthupetai, dhanush, ah, yarellam, pathinga]",puthupeta dhanush ah yarellam pathing
3,"dhanush oda character ,puthu sa erukay , mass ta",positive,dhanush oda character puthu sa erukay mass ta,dhanush oda character puthu sa erukay mass ta,dhanush oda character puthu sa erukay mass ta,"[dhanush, oda, character, puthu, sa, erukay, m...","[dhanush, oda, character, puthu, sa, erukay, m...",dhanush od charact puthu sa erukay mass ta
4,vera level ippa pesungada mokka nu thalaivaaaaaa,positive,vera level ippa pesungada mokka nu thalaivaaaaaa,vera level ippa pesungada mokka nu thalaivaaaaaa,vera level ippa pesungada mokka nu thalaivaaaaaa,"[vera, level, ippa, pesungada, mokka, nu, thal...","[vera, level, ippa, pesungada, mokka, nu, thal...",ver level ipp pesungad mokk nu thalaivaaaaa
...,...,...,...,...,...,...,...,...
11330,yuvan shankar raja anna fan's like here...,positive,yuvan shankar raja anna fan like here,yuvan shankar raja anna fan like here,yuvan shankar raja anna fan like here,"[yuvan, shankar, raja, anna, fan, like, here]","[yuvan, shankar, raja, anna, fan, like]",yuv shank raj ann fan lik
11331,a masterpiece best revenge film i’ve ever scene,positive,a masterpiece best revenge film i ve ever scene,a masterpiece best revenge film i ve ever scene,a masterpiece best revenge film i ve ever scene,"[a, masterpiece, best, revenge, film, i, ve, e...","[masterpiece, best, revenge, film, ever, scene]",masterpiec best reveng film ev scen
11332,enna pa thala ya kamiya than katringa,positive,enna pa thala ya kamiya than katringa,enna pa thala ya kamiya than katringa,enna pa thala ya kamiya than katringa,"[enna, pa, thala, ya, kamiya, than, katringa]","[enna, pa, thala, ya, kamiya, katringa]",enn pa thal ya kamiy katring
11333,r a a s h i k h a n n a,unknown_state,r a a s h i k h a n n a,r a a s h i k h a n n a,r a a s h i k h a n n a,"[r, a, a, s, h, i, k, h, a, n, n, a]",[],


In [16]:
test_set = datacleen(test_set)
test_set

Stemming And Lemmatizing: 100%|██████████| 1257/1257 [00:00<00:00, 5240.10it/s]


,text,category,take_data_to_shower,remove_emoticon,remove_punc,tokenize,remove_stop_words,stem_and_lem
0,daily likes & views pakka vanthavaga ellarukum...,positive,daily likes views pakka vanthavaga ellarukum...,daily likes views pakka vanthavaga ellarukum...,daily likes views pakka vanthavaga ellarukum...,"[daily, likes, views, pakka, vanthavaga, ellar...","[daily, likes, views, pakka, vanthavaga, ellar...",dai lik view pakk vanthavag ellaruk vanakkam
1,25 k dislikes ethuku da intha trailerku poi ap...,negative,k dislikes ethuku da intha trailerku poi ap...,k dislikes ethuku da intha trailerku poi ap...,k dislikes ethuku da intha trailerku poi ap...,"[k, dislikes, ethuku, da, intha, trailerku, po...","[dislikes, ethuku, da, intha, trailerku, poi, ...",dislik ethuku da inth trailerku poi apd ungalu...
3,it looks like hindi movie amitab bachan,positive,it looks like hindi movie amitab bachan,it looks like hindi movie amitab bachan,it looks like hindi movie amitab bachan,"[it, looks, like, hindi, movie, amitab, bachan]","[looks, like, hindi, movie, amitab, bachan]",look lik hind movy amitab bach
4,thalaivarukku nejamavey vayasaagiduchu... siva...,positive,thalaivarukku nejamavey vayasaagiduchu siva...,thalaivarukku nejamavey vayasaagiduchu siva...,thalaivarukku nejamavey vayasaagiduchu siva...,"[thalaivarukku, nejamavey, vayasaagiduchu, siv...","[thalaivarukku, nejamavey, vayasaagiduchu, siv...",thalaivarukku nejamavey vayasaagiduchu sivaj t...
5,thala nu sollu thala nemirinthu nillu,positive,thala nu sollu thala nemirinthu nillu,thala nu sollu thala nemirinthu nillu,thala nu sollu thala nemirinthu nillu,"[thala, nu, sollu, thala, nemirinthu, nillu]","[thala, nu, sollu, thala, nemirinthu, nillu]",thal nu sollu thal nemirinthu nillu
...,...,...,...,...,...,...,...,...
1255,sappypathy vijayamma bigilu ummpi get more dis...,negative,sappypathy vijayamma bigilu ummpi get more dis...,sappypathy vijayamma bigilu ummpi get more dis...,sappypathy vijayamma bigilu ummpi get more dis...,"[sappypathy, vijayamma, bigilu, ummpi, get, mo...","[sappypathy, vijayamma, bigilu, ummpi, get, di...",sappypathy vijayamm bigilu ummp get dislik tar...
1256,stylea erukana ha ha naturally,positive,stylea erukana ha ha naturally,stylea erukana ha ha naturally,stylea erukana ha ha naturally,"[stylea, erukana, ha, ha, naturally]","[stylea, erukana, ha, ha, naturally]",style erukan ha ha nat
1257,style la irukana hahaha mass dialogue,negative,style la irukana hahaha mass dialogue,style la irukana hahaha mass dialogue,style la irukana hahaha mass dialogue,"[style, la, irukana, hahaha, mass, dialogue]","[style, la, irukana, hahaha, mass, dialogue]",styl la irukan hahah mass dialog
1258,like for lokesh kanagaraj thalapathy 64 director,unknown_state,like for lokesh kanagaraj thalapathy director,like for lokesh kanagaraj thalapathy director,like for lokesh kanagaraj thalapathy director,"[like, for, lokesh, kanagaraj, thalapathy, dir...","[like, lokesh, kanagaraj, thalapathy, director]",lik lokesh kanagaras thalapathy direct


# test 1


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_set['stem_and_lem'])
y_train = train_set['category']

print(X_train)
print(y_train)
print("--------------------------------------------------------------------------------")

X_test = vectorizer.transform(test_set['stem_and_lem'])
y_test = test_set['category']

print(X_test)
print(y_test)


  (0, 8906)	0.3607377870793489
  (0, 5528)	0.20803138610603505
  (0, 8367)	0.6047779759519744
  (0, 453)	0.3039024666865819
  (0, 5457)	0.5524421363720533
  (0, 12062)	0.25156662710666083
  (1, 13855)	0.37863044220083725
  (1, 7913)	0.5722383963196445
  (1, 12727)	0.39205201196472156
  (1, 8523)	0.38364565359641356
  (1, 6315)	0.4337432156124812
  (1, 12062)	0.2004002527879125
  (2, 8476)	0.5104873207211479
  (2, 13840)	0.43953644414091086
  (2, 2004)	0.3179922001597301
  (2, 9551)	0.6144284761829207
  (2, 453)	0.2599404123025423
  (3, 11134)	0.3558937297641465
  (3, 5970)	0.15944816054959154
  (3, 2679)	0.48171035738943674
  (3, 9886)	0.39098939048583625
  (3, 9549)	0.426085051207526
  (3, 1609)	0.3831843815195372
  (3, 7188)	0.2691962194590797
  (3, 2004)	0.24930507345234731
  :	:
  (9961, 14095)	0.4369957197350149
  (9961, 727)	0.3891372229478061
  (9961, 2890)	0.27924059412077723
  (9961, 5528)	0.23170199278012035
  (9962, 6001)	0.5311118091765067
  (9962, 9802)	0.5013255846738454


In [19]:

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report



classifier = SVC(kernel='linear', C=1.0)

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
print("Confusion Report:")
conf_report = classification_report(y_test, y_pred)
print(conf_report)


Accuracy: 0.7894265232974911
Confusion Matrix:
[[ 24   2 138   0]
 [  2  13  14   0]
 [ 11   2 842   0]
 [  0   0  66   2]]
Confusion Report:
               precision    recall  f1-score   support

     negative       0.65      0.15      0.24       164
    not-tamil       0.76      0.45      0.57        29
     positive       0.79      0.98      0.88       855
unknown_state       1.00      0.03      0.06        68

     accuracy                           0.79      1116
    macro avg       0.80      0.40      0.44      1116
 weighted avg       0.78      0.79      0.73      1116

